# **1. LogisticRegression model**

### **1-1. 데이터 확인**

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(3)
tf.random.set_seed(3)

pd_train = pd.read_csv('train.csv')
pd_test = pd.read_csv('test.csv')
pd_submit = pd.read_csv('gender_submission.csv')

In [ ]:
pd_train

*   PassengerId : 승객 번호
*   Survived : 생존 여부 (NO=0, YES=1) -> 테스트 데이터로 이 생존 여부 예측하는 것!
*   Pclass : 티켓 등급 (1=1st, 2=2nd, 3=3rd) ★
*   Name : 이름 (의미있는 정보 뽑아서 숫자로 바꿔야 함)
*   Sex : 성별 (숫자로 바꿔야 함)
*   Age : 나이 (적당한 기준 세워서 간단한 숫자로 바꿔보자)
*   SibSp : 사촌이나 배우자가 동승 했는지
*   Parch : 부모나 자식이 동승 했는지
*   Ticket : 티켓 넘버
*   Fare : 요금
*   Cabin	: 선실 넘버 (NAN : Not a Number 주의)
*   Embarked : 어떤 선착장에서 탔는지

In [ ]:
pd_train.info() #12개의 컬럼과, 891개의 승객 정보
                #Age는 714개의 정보만 존재
                #Cabin은 204개의 정보만 존재
                #Nan은 파일이 비어있음

In [ ]:
pd_test.info() #11개의 컬럼과 418개의 승객 정보
               #빠진 하나의 컬럼은 생존 여부를 나타내는 'Survived'
               #마찬가지로 Age하고 Cabin은 정보가 좀 빠져 있다!

In [ ]:
pd_train.isnull().sum() #Age랑 Cabin은 많이 빠져 있어서 보정 필요하다!
                        #Embarked도 2개는 빠져 있다.

In [ ]:
pd_test.isnull().sum() #여기도 역시 Age랑 Cabin은 많이 빠져 있다.
                       #Fare도 1개 빠져 있다.

### **1-2. 데이터 시각화**

In [ ]:
sns.countplot(data=pd_train, x="Survived", hue="Pclass", order= [1, 0]) #티켓 등급별(1등석, 2등석, 3등석) 생존 여부(생존=1, 사망=0)
                                                                        #생존률은 1등석 승객이 높고, 사망률은 3등석 승객이 높음

In [ ]:
sns.countplot(data=pd_train, x="Survived", hue="Sex", order= [1, 0]) #성별 별 생존 여부(생존=1, 사망=0)
                                                                     #생존률은 여성이 높고, 사망률은 남성이 높음

In [ ]:
grid = sns.FacetGrid(pd_train, hue='Survived', size=4)
grid.map(plt.hist, 'Age', alpha=.6, rwidth=0.8, bins=[0,10,20,30,40,50,60,70,80,90,100])

In [ ]:
sns.countplot(data=pd_train, x="Survived", hue="SibSp", order= [1, 0]) 
#사촌이나 배우자 동승 별 생존 여부(생존=1, 사망=0)

In [ ]:
sns.countplot(data=pd_train, x="Survived", hue="Parch", order= [1, 0]) 
#부모나 자식 동승 별 생존 여부(생존=1, 사망=0)

In [ ]:
sns.countplot(data=pd_train, x="Survived", hue="Embarked", order= [1, 0]) 
#선착장 별 생존 여부(생존=1, 사망=0)

### **1-3. 데이터 전처리**

< To Do List >
1. 삭제할 데이터 지우기
2. Name : 이름 (의미있는 정보 뽑아서 숫자로 바꿔야 함)
3. Sex : 성별 (숫자로 바꿔야 함)
4. Age : 나이  (빠져있는 값 많았음, 적당한 기준 세워서 간단한 숫자로 바꿔보자)
5. Embarked : 빠져있는 데이터 어떻게 처리할지?

In [ ]:
train_test_data = [pd_train, pd_test] #전처리 한번에 하기 위해 잠깐 학습셋이랑 테스트셋 합침

**1. 삭제할 데이터 지우기**

*   PassengerId : 필요 없어
*   SibSp랑 Parch : 묶어서 가족으로 처리해봤지만 지우는게 더 점수 높았음
*   Ticket : 필요 없어
*   Fare : 요금을 아주 적게 낸(15이하?) 사람들 사망률 높길래 따로 처리 해봤지만, 없애는게 더 점수 높았음
*   Cabin : 알파벳 대문자만 따서 숫자로 바꾸고, NaN값도 Pclass랑 비교해서 전부 넣어주기까지 해봤는데 그냥 없애는게 더 점수 높았음

In [ ]:
pd_train.drop('PassengerId', axis=1, inplace=True)
#pd_test.drop('PassengerId', axis=1, inplace=True)?

pd_train.drop('SibSp', axis=1, inplace=True)
pd_test.drop('SibSp', axis=1, inplace=True)

pd_train.drop('Parch', axis=1, inplace=True)
pd_test.drop('Parch', axis=1, inplace=True)

pd_train.drop('Ticket', axis=1, inplace=True)
pd_test.drop('Ticket', axis=1, inplace=True)

pd_train.drop('Fare', axis=1, inplace=True)
pd_test.drop('Fare', axis=1, inplace=True)

pd_train.drop('Cabin', axis=1, inplace=True)
pd_test.drop('Cabin', axis=1, inplace=True)

**2. Name : 이름 (의미있는 정보 뽑아서 숫자로 바꿔야 함)**

In [ ]:
for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False) #대문자로 시작하여 소문자로 나열되며 .을 만나면 멈추고 Title에 저장

In [ ]:
pd_train['Title'].value_counts() #각각의 value 빈도 측정

In [ ]:
#특징이 뚜렷한 Mr,Miss,Mrs만 0,1,2로 놓고 나머지는 3으로!
title_mapping = {"Mr": 0, "Miss": 1, "Mrs": 2, 
                 "Master": 3, "Dr": 3, "Rev": 3, "Col": 3, "Major": 3, "Mlle": 3,"Countess": 3, "Ms": 3, "Lady": 3, "Jonkheer": 3, "Don": 3, "Dona" : 3, "Mme": 3,"Capt": 3,"Sir": 3 }
for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].map(title_mapping)

In [ ]:
sns.countplot(data=pd_train, x="Survived", hue="Title", order= [1, 0]) 
#Mr에 비해 Miss 하고 Mrs가 생존률이 높음

In [ ]:
#이름 속성 삭제
pd_train.drop('Name', axis=1, inplace=True)
pd_test.drop('Name', axis=1, inplace=True)

In [ ]:
pd_train.head()

**3. Sex : 성별 (숫자로 바꿔야 함)**

In [ ]:
#남자 0으로 여자 1로 #LabelEncoder써도 되는데 train, test 두번 해줘야 함
sex_mapping = {"male": 0, "female": 1}
for dataset in train_test_data:
    dataset['Sex'] = dataset['Sex'].map(sex_mapping)

In [ ]:
pd_train.head()

**4. Age : 나이 (빠져있는 값 많았음, 적당한 기준 세워서 간단한 숫자로 바꿔보자)**

In [ ]:
#Age에서 빠져있는 값이 있으면, Title정보를 보고 그 정보에 속하는 집단의 중앙값을 나이로 넣자!
pd_train["Age"].fillna(pd_train.groupby("Title")["Age"].transform("median"), inplace=True) #mean해도 점수차이는 없음
pd_test["Age"].fillna(pd_test.groupby("Title")["Age"].transform("median"), inplace=True)

In [ ]:
pd_train.isnull().sum() #Age값 전부 채워진 것 확인

In [ ]:
#Age도 적당한 기준으로 나눠보자
g = sns.FacetGrid(pd_train, col='Survived')
g.map(plt.hist, 'Age', bins=20);
plt.xlim(0, 30)
#plt.xlim(16, 50)
#plt.xlim(30, 40)
#plt.xlim(36, 80)

In [ ]:
for dataset in train_test_data:
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 26), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 26) & (dataset['Age'] <= 30), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 30) & (dataset['Age'] <= 36), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 36, 'Age'] = 4

**5. Embarked : 빠져있는 데이터 어떻게 처리할지?**

In [ ]:
#Embarked는 train 데이터에서 2개 빠져 있었음
sns.countplot(data=pd_train, x="Pclass", hue="Embarked")

In [ ]:
for dataset in train_test_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

In [ ]:
pd_train.isnull().sum() #Embarked값 전부 채워진 것 확인

In [ ]:
embarked_mapping = {"S": 0, "C": 1, "Q": 2}
for dataset in train_test_data:
    dataset['Embarked'] = dataset['Embarked'].map(embarked_mapping)

In [ ]:
pd_train.head()

### **1-4. 모델 생성 및 학습과 테스트**

In [ ]:
#학습데이터 슬라이싱
x_train = pd_train.iloc[:,1:]
y_train = pd_train.iloc[:,:1]
#테스트데이터 슬라이싱
x_test = pd_test.iloc[:,1:]

In [ ]:
#로지스틱회귀 쓰려면 데이터 정규화(스케일링)! 각 feature의 평균을 0, 분산을 1로 변경, 모든 특성들이 같은 스케일을 갖게 됨
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train, y_train)

In [ ]:
y_test = model.predict(x_test).flatten()

### **1-5. 예측 결과 저장**

In [ ]:
for i in range(len(pd_submit)):
  pd_submit['Survived'][i] = np.round(y_test[i]).astype(int)
  #pd_submit['Survived']= pd_submit['Survived'].astype(int)
pd_submit

In [ ]:
pd_submit.to_csv("Titanic_result.csv", index=False,header=True)

In [38]:
pd_submit.to_csv("Titanic_result.csv", index=False,header=True)

# **2. Sequential model**

### **2-1.데이터 개요 파악**

In [ ]:
#!pip install kaggle
#from google.colab import files
#files.upload()

In [ ]:
#!mkdir -p ~/.kaggle
#!cp kaggle.json ~/.kaggle/
#!chmod 600 ~/.kaggle/kaggle.json
#!ls -lha kaggle.json

In [ ]:
#!kaggle competitions download -c titanic

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
import random
import os

In [ ]:
# seed 설정
np.random.seed(3)
tf.random.set_seed(3)

pd_train = pd.read_csv('train.csv')
pd_test = pd.read_csv('test.csv')

pd_train

**문제: 사람들의 생존여부 분류**
* sibsp: 사촌, 형제자매
* parch : 부모님, 자녀
* Cabin: Cabin number
* Embarked: Port of Embarkation
* Ticket: 티켓번호

In [ ]:
pd_train.info()

- age, cabin, embarked 속성에 결측치 존재  
- Name ,Sex ,Ticket 등 여러 속성값 자료형 object라 전처리 필요      
- 중요한 class인 survived 가 1열        

### **2-2. 히트맵으로 데이터 상관관계 파악**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.pairplot(pd_train,hue='Survived')
plt.show()

페어플롯에서는 pclass랑 fare이 산관관계가 높다는 것만 파악

### 생존여부와 다른 속성들간의 관계 파악

#### Pclass 와 Survived

In [ ]:
sns.barplot(x='Pclass',y='Survived', data=pd_train)

- 티켓 클래스가 낮을 수록 생존률 높음

#### Sex 와 Survived

In [ ]:
sns.barplot(x='Sex',y='Survived', data=pd_train)

- 여자의 생존율이 월등히 높음( 여자먼저 구출한 듯)

#### Age 와 Survived

In [ ]:
g=sns.FacetGrid(pd_train,col='Survived')
g.map(plt.hist,'Age',bins=10)
plt.show()

- 특이한 점은 10세 이하 아이들의 생존확률이 눈에 띄게 높다 ( 아마 아이들 먼저 구출하려고 해서 그런 듯)

#### SibSp  와 Survived

In [ ]:
sns.barplot(x='SibSp',y='Survived', data=pd_train)

- 큰 연관성은 안보인다.

#### Parch 와 Survived

In [ ]:
sns.barplot(x='Parch',y='Survived', data=pd_train)

- 부모,자녀의 수가 1,2,3 명 있는 사람이 없는 사람보다는 생존율이 높다( 아마 부모,자녀를 지키려고 생존욕구가 더 클 것으로 생각)
- 하지만 부모자녀가 5명인 사람들의 생존율이 제일 낮음( 사람이 너무많아 생존에 불리했을 것으로 생각)

#### Fare 와 Survived

In [ ]:
g=sns.FacetGrid(pd_train,col='Survived')
g.map(plt.hist,'Fare',bins=20)
plt.show()

- 낮은요금의 승객보다 높은 요금의 승객의 생존율이 더 높다

#### Embarked 와 Survived

In [ ]:
sns.barplot(x='Embarked',y='Survived', data=pd_train)

- c 포트에서 탑승한 사람이 생존율 높음

### 상자화 한 후 시각화 재확인

In [ ]:
def make_bins(d, col, factor=2):
    rounding = lambda x: np.around(x / factor)
    d[col] = d[col].apply(rounding)
    return d

t = make_bins(pd_train.copy(True), 'Age', 2)
sns.barplot(x="Age", y="Survived", data=t);

In [ ]:
g = sns.FacetGrid(pd_train, col='Survived')
g.map(plt.hist, 'Age', bins=20);

In [ ]:
t = make_bins(pd_train, 'Fare', 10)
sns.barplot(x="Fare", y="Survived", data=t);

In [ ]:
sns.barplot(x="Embarked", y="Survived", data=pd_train);

### 분석 결과

- 생존 속성과 크게 연관이 있는것은 pclass, sex, age, embarked로 보임
- name은 id로 대체되서 누락시킬 에정
- SibSp,Parch 는 나중에 점수 개선시 넣어볼 예정
- Fare는 pclass와 같다고 생각해 누락
- cabin 은 결측치가 너무많아 누락

### 학습 데이터 전처리

#### 주요 속성만 가져오기

In [ ]:
train_set = pd_train[['Pclass','Sex','Age','Embarked','Survived']]
train_set

#### age 결측치 처리 - 1 (sub 파일)

In [ ]:
#sub=pd_train[["Pclass", "Sex","Embarked","Survived"]]
sub=pd_train[["Pclass", "Sex", "Age","Embarked","Survived"]]
sub

In [ ]:
sub.isnull().sum()

In [ ]:
sub['Embarked'] = sub['Embarked'].fillna(value='C')

In [ ]:
sub.isnull().sum()

In [ ]:
g = sns.FacetGrid(pd_train, col='Pclass')
g.map(plt.hist, 'Age', bins=20);

In [ ]:
g = sns.FacetGrid(pd_train, col='Sex')
g.map(plt.hist, 'Age', bins=20);

In [ ]:
avg_age = sub.groupby(['Pclass', 'Sex'], as_index=False).mean()

avg_age

In [ ]:
sub.loc[sub['Age'].isnull()&(sub['Pclass'] == 1)&(sub['Sex'] == 'female'),'Age'] = avg_age['Age'][0]
sub.loc[sub['Age'].isnull()&(sub['Pclass'] == 1)&(sub['Sex'] == 'male'),'Age' ]= avg_age['Age'][1]

sub.loc[sub['Age'].isnull()&(sub['Pclass'] == 2)&(sub['Sex'] == 'female'),'Age'] = avg_age['Age'][2]
sub.loc[sub['Age'].isnull()&(sub['Pclass'] == 2)&(sub['Sex'] == 'male'),'Age' ]= avg_age['Age'][3]

sub.loc[sub['Age'].isnull()&(sub['Pclass'] == 3)&(sub['Sex'] == 'female'),'Age'] = avg_age['Age'][4]
sub.loc[sub['Age'].isnull()&(sub['Pclass'] == 3)&(sub['Sex'] == 'male'),'Age' ]= avg_age['Age'][5]

In [ ]:
# 확인
g = sns.FacetGrid(sub, col='Pclass')
g.map(plt.hist, 'Age', bins=20);

In [ ]:
sub.isnull().sum()

#### 문자 숫자로 인코딩 후 모델링 (sub)

In [ ]:
from sklearn.preprocessing import LabelEncoder

e=LabelEncoder()
e.fit(sub['Sex'])
sub['Sex']=e.transform(sub['Sex'])

e.fit(sub['Embarked'])
sub['Embarked']=e.transform(sub['Embarked'])

In [ ]:
x_train = sub.iloc[:,:-1]
y_train = sub.iloc[:,-1]
x_train

In [ ]:
y_train

In [ ]:
model=Sequential()
model.add(Dense(64,input_dim=4,activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#ad = optimizers.Adam(lr = 0.001)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
# from keras.callbacks import ModelCheckpoint, EarlyStopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=70)

In [ ]:
hist=model.fit(x_train, y_train, epochs=30000, batch_size=32, validation_split=0.2, callbacks=[early_stopping_callback])
#hist=model.fit(x_train, y_train, epochs=170, batch_size=32, validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(hist.history['loss'], 'b-', label='loss')
plt.plot(hist.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(hist.history['accuracy'], 'b-', label='accuracy')
plt.plot(hist.history['val_accuracy'], 'r--', label='val_accuracy')
plt.xlabel('Epoch')
plt.legend()

In [ ]:
pd_test

In [ ]:
test_data = pd_test[["Pclass", "Sex", "Age", "Embarked"]]
test_data.isnull().sum()

In [ ]:
test_data.loc[test_data['Age'].isnull()&(test_data['Pclass'] == 1)&(test_data['Sex'] == 'female'),'Age'] = avg_age['Age'][0]
test_data.loc[test_data['Age'].isnull()&(test_data['Pclass'] == 1)&(test_data['Sex'] == 'male'),'Age' ]= avg_age['Age'][1]

test_data.loc[test_data['Age'].isnull()&(test_data['Pclass'] == 2)&(test_data['Sex'] == 'female'),'Age'] = avg_age['Age'][2]
test_data.loc[test_data['Age'].isnull()&(test_data['Pclass'] == 2)&(test_data['Sex'] == 'male'),'Age' ]= avg_age['Age'][3]

test_data.loc[test_data['Age'].isnull()&(test_data['Pclass'] == 3)&(test_data['Sex'] == 'female'),'Age'] = avg_age['Age'][4]
test_data.loc[test_data['Age'].isnull()&(test_data['Pclass'] == 3)&(test_data['Sex'] == 'male'),'Age' ]= avg_age['Age'][5]

In [ ]:
test_data.isnull().sum()

In [ ]:
test_data['Embarked'] = test_data['Embarked'].fillna(value='C')

e=LabelEncoder()
e.fit(test_data['Sex'])
test_data['Sex']=e.transform(test_data['Sex'])
e.fit(test_data['Embarked'])
test_data['Embarked']=e.transform(test_data['Embarked'])

test_data

In [ ]:
y_test = model.predict(test_data)
y_test

In [ ]:
len(y_test)

In [ ]:
pd_submit = pd.read_csv('gender_submission.csv')
print(pd_submit)

In [ ]:
for i in range(len(pd_submit)):
  pd_submit['Survived'][i] = np.round(y_test[i]).astype(int)
# pd_submit['Survived']= pd_submit['Survived'].astype(int)
pd_submit

In [ ]:
pd_submit.to_csv("resul1t.csv", index=False,header=True)

In [ ]:
#!kaggle competitions submit -c titanic -f resul1t.csv -m "Message"

#### 결측치 처리 - 2 (drop)

* drop(['Cabin'], axis=1) # 행이나 열 없애기
* dropna(inplace=True) #nan 없애기, inplace는 실제 데이터값 채우기

In [ ]:
train_set.dropna(inplace=True) #nan 없애기, inplace는 실제 데이터값 채우기
train_set

결측치 존재 행 삭제됨

#### 문자 숫자로 인코딩 후 모델링 (원파일)

자료형이 object 인 Sex,Embarked 숫자로 인코딩 해줘야 함

sex 인코딩

In [ ]:
from sklearn.preprocessing import LabelEncoder

e= LabelEncoder()
e.fit(train_set['Sex'])
train_set['Sex']=e.transform(train_set['Sex'])
train_set

Embarked 인코딩

In [ ]:
e= LabelEncoder()
e.fit(train_set['Embarked'])
train_set['Embarked']=e.transform(train_set['Embarked'])
train_set

In [ ]:
x_train = train_set.iloc[:,:4]
y_train = train_set.iloc[:,4]
print(x_train,y_train)

In [ ]:
x_train = train_set.iloc[:,:4]
y_train = train_set.iloc[:,4]
print(x_train,y_train)

dropout 성능 떨어짐

In [ ]:
from keras.layers import Flatten, Dropout, BatchNormalization

model = Sequential()
model.add(Dense(32,input_dim=4,activation='relu'))
#model.add(Dropout(0.1))  
model.add(Dense(16,activation='relu'))
#model.add(Dropout(0.1))  
model.add(Dense(8,activation='relu'))
#model.add(Dropout(0.1))  
model.add(Dense(1,activation='sigmoid')) 

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

생존 분류-> sigmoid 함수 사용

여러 제출 결과 이 모델에선 최상의 에포크 225와 배치사이즈 임

In [ ]:
model.fit(x_train,y_train,epochs=225,batch_size=10,validation_split=0.25)

test셋 전처리

In [ ]:
pd_test

In [ ]:
test_set = pd_test[['Pclass','Sex','Age','Embarked']]
test_set

test셋 결측치 처리

In [ ]:
test_set=test_set.fillna(method='ffill') # 자신보다 앞에있는 값으로 nan 채움
#test_set.fillna(method='bfill') # 자신보다 뒤에있는 값으로 nan 채움
test_set

테스트셋 인코딩

In [ ]:
from sklearn.preprocessing import LabelEncoder

e=LabelEncoder()
e.fit(test_set['Sex'])
test_set['Sex'] = e.transform(test_set['Sex'])
test_set

In [ ]:
e=LabelEncoder()
e.fit(test_set['Embarked'])
test_set['Embarked'] = e.transform(test_set['Embarked'])
test_set

예측 값 생성

In [ ]:
y_test = model.predict(test_set)
y_test

In [ ]:
len(y_test)

In [ ]:
y_test = y_test.flatten() # [[1][2]]를 [1,2] 로
y_test

예측값 생존 0,1 두가지로 바꾸기

In [ ]:
y_test = np.round(y_test).astype(int)
y_test

In [ ]:
pd_submit = pd.read_csv('gender_submission.csv')
pd_submit

In [ ]:
submit = pd.DataFrame({'PassengerId':pd_submit['PassengerId'],'Survived':y_test})
submit

In [ ]:
submit.to_csv("submit8.csv",index=False,header=True)